<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/CCAP_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

In [5]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode

# Additional code can be added here if needed

In [6]:
directory_path = '/content/image_subsets'
os.makedirs(directory_path, exist_ok=True)

In [7]:
zip_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.zip')
extract_dir = '/content/image_subsets' # destination directory
for zip_file in zip_files:
    base_name = os.path.basename(zip_file)[:-4]
    unzip_dir = os.path.join(extract_dir, base_name)
    os.makedirs(unzip_dir, exist_ok=True)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)

In [8]:
csv_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.csv')
csv_list = []
for csv_file in csv_files:
    base_name = os.path.basename(csv_file)[:-4]
    csv_dir = os.path.join(extract_dir, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)
    index_csv = pd.concat(csv_list, ignore_index=True)

In [9]:
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).size().reset_index(name='Count')
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).agg({'percent': 'mean'}).reset_index()

In [10]:
pivot_df = unique_rows_df.pivot(index=['tif_name', 'subset'], columns='label', values='percent').reset_index().fillna(0)

In [11]:
merged_df = index_csv.merge(pivot_df, on=['tif_name', 'subset'], how='left')
merged_df.drop(columns=['label'], inplace=True)

In [12]:
sum_df = merged_df.drop_duplicates(subset=['tif_name', 'subset'])

In [14]:
total_rows = len(sum_df)
train_fraction = 0.9 # modify this to set the training percentage
train_rows = int(total_rows * train_fraction)
val_rows = total_rows - train_rows

random_assignment = np.array([0] * train_rows + [1] * val_rows)
np.random.seed(42)
np.random.shuffle(random_assignment)
sum_df['random_split'] = random_assignment

<ipython-input-14-66bd3969d95d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['random_split'] = random_assignment


In [15]:
agg_df = sum_df.groupby(['random_split']).agg({13: 'mean',14:'mean',15: 'mean',16:'mean',17: 'mean',18:'mean'}).reset_index()
print(agg_df)

   random_split        13        14        15        16        17        18
0             0  0.152032  0.034274  0.011877  0.001600  0.001068  0.001292
1             1  0.145450  0.036990  0.009983  0.002517  0.001619  0.001482


In [16]:
sum_df['Images_path']="/content/image_subsets/"+sum_df['subset']+'/Images/'+sum_df['tif_name']
sum_df['Labels_path']="/content/image_subsets/"+sum_df['subset']+'/Labels/'+sum_df['tif_name']

<ipython-input-16-734faffeae8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Images_path']="/content/image_subsets/"+sum_df['subset']+'/Images/'+sum_df['tif_name']
<ipython-input-16-734faffeae8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Labels_path']="/content/image_subsets/"+sum_df['subset']+'/Labels/'+sum_df['tif_name']


In [18]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,...,subset,13,14,15,16,17,18,random_split,Images_path,Labels_path
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,0.000919,...,Kauai10_subset,0.000919,0.000000,0.000000,0.000000,0.000000,0.000000,0,/content/image_subsets/Kauai10_subset/Images/8...,/content/image_subsets/Kauai10_subset/Labels/8...
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,0.033161,...,Kauai10_subset,0.033161,0.000000,0.000000,0.000000,0.000000,0.000000,1,/content/image_subsets/Kauai10_subset/Images/9...,/content/image_subsets/Kauai10_subset/Labels/9...
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,0.000130,...,Kauai10_subset,0.000130,0.000137,0.003502,0.000000,0.000000,0.000000,0,/content/image_subsets/Kauai10_subset/Images/1...,/content/image_subsets/Kauai10_subset/Labels/1...
5,15361,1025,14849,513,73,453579.566028,454808.362434,2.457574e+06,2.458802e+06,0.001663,...,Kauai10_subset,0.000000,0.000000,0.001663,0.000000,0.000000,0.000000,0,/content/image_subsets/Kauai10_subset/Images/1...,/content/image_subsets/Kauai10_subset/Labels/1...
6,9217,1537,8705,1025,104,438833.962924,440062.791964,2.456345e+06,2.457574e+06,0.007381,...,Kauai10_subset,0.007381,0.000000,0.000000,0.000000,0.000000,0.000000,0,/content/image_subsets/Kauai10_subset/Images/8...,/content/image_subsets/Kauai10_subset/Labels/8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,17921,15361,17409,14849,1079,308475.532176,309704.324054,2.156737e+06,2.157966e+06,0.012070,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.012070,0.000416,0.001926,0,/content/image_subsets/Hawaii_2010_008_subset/...,/content/image_subsets/Hawaii_2010_008_subset/...
6236,3585,15873,3073,15361,1087,274069.129264,275297.911026,2.155509e+06,2.156737e+06,0.000507,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000507,0.000000,0.000000,0.000000,0,/content/image_subsets/Hawaii_2010_008_subset/...,/content/image_subsets/Hawaii_2010_008_subset/...
6237,2049,16897,1537,16385,1156,270382.720297,271611.486128,2.153051e+06,2.154280e+06,0.000847,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000847,0.000000,0.000000,0.000000,0,/content/image_subsets/Hawaii_2010_008_subset/...,/content/image_subsets/Hawaii_2010_008_subset/...
6238,17409,16897,16897,16385,1186,307246.720617,308475.512807,2.153051e+06,2.154280e+06,0.000889,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.000000,0.000000,0.000889,0,/content/image_subsets/Hawaii_2010_008_subset/...,/content/image_subsets/Hawaii_2010_008_subset/...
